<a href="https://colab.research.google.com/github/seelokyen/bottledetectionCNNmodel/blob/main/CP2_Train_CNN_Model_BaselineYOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/MyDrive/bottles_dataset/data.zip /content

Mounted at /content/gdrive


In [2]:
# Unzip images to a custom data folder
!unzip -q /content/data.zip -d /content/

In [3]:
import os
import random
import shutil

# Get all image filenames from the full dataset
image_dir = '/content/data/images'
label_dir = '/content/data/labels'
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]

# Shuffle full dataset
random.shuffle(image_files)

# Define split percentages
train_pct, val_pct, test_pct = 0.7, 0.15, 0.15
total = len(image_files)
train_end = int(train_pct * total)
val_end = train_end + int(val_pct * total)

# Setup directories
base = '/content/data'
splits = ['train', 'val', 'test']
for split in splits:
    os.makedirs(f'{base}/{split}/images', exist_ok=True)
    os.makedirs(f'{base}/{split}/labels', exist_ok=True)

# Split and copy
for i, img_file in enumerate(image_files):
    split = 'train' if i < train_end else 'val' if i < val_end else 'test'
    label_file = os.path.splitext(img_file)[0] + '.txt'
    shutil.copy(os.path.join(image_dir, img_file), f'{base}/{split}/images/{img_file}')
    shutil.copy(os.path.join(label_dir, label_file), f'{base}/{split}/labels/{label_file}')

print(f"Finished splitting full dataset into train/val/test in '{base}'")

Finished splitting full dataset into train/val/test in '/content/data'


In [4]:
# Create data.yaml file
data_yaml = """
path: /content/data
train: train/images
val: val/images
test: test/images

nc: 1
names: ['bottle']
"""

with open('/content/data/data.yaml', 'w') as f:
    f.write(data_yaml)

print("Finished creating data.yaml file")

Finished creating data.yaml file


In [5]:
!pip install ultralytics
print("Finished installing ultralytics")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [6]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # or yolov8s.pt for slightly larger

# Print baseline hyperparameters
print("Model hyperparameters")
print(model.args)
print("Done printing model hyperparameters")

model.train(
    data='/content/data/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    name='baseline_yolov8',
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 74.0MB/s]


Model hyperparameters
{'task': 'detect', 'data': 'coco.yaml', 'imgsz': 640, 'single_cls': False, 'model': 'yolov8n.pt'}
Done printing model hyperparameters
Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=F

100%|██████████| 755k/755k [00:00<00:00, 14.5MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

      50/50         4G     0.7836     0.3857     0.9947         21        640: 100%|██████████| 628/628 [00:53<00:00, 11.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 68/68 [00:08<00:00,  7.59it/s]


                   all       2151       8175      0.945      0.927      0.967      0.763

50 epochs completed in 0.897 hours.
Optimizer stripped from runs/detect/baseline_yolov8/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/baseline_yolov8/weights/best.pt, 6.2MB

Validating runs/detect/baseline_yolov8/weights/best.pt...
Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 68/68 [00:11<00:00,  6.02it/s]


                   all       2151       8175      0.948      0.925      0.967      0.763
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/baseline_yolov8


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d4dfdea5f90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [7]:
model.predict(source='/content/data/test/images', save=True)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2152 /content/data/test/images/-106_jpg.rf.278d36795cc1bfcf971a40149f351a1c.jpg: 640x640 5 bottles, 7.7ms
image 2/2152 /content/data/test/images/-124_jpg.rf.ae84f03126700a9776b5c1e083f14125.jpg: 640x640 1 bottle, 8.4ms
image 3/2152 /content/data/test/images/-124_jpg.rf.c5851eeec894c732e6986d328dfed23d.jpg: 640x640 1 bottle, 9.6ms
image 4/2152 /content/data/test/images/-136_jpg.rf.51966909228f1477d6328e44a858a09c.jpg: 640x640 2 bottles, 9.3ms
im

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bottle'}
 obb: None
 orig_img: array([[[145, 151, 156],
         [145, 151, 156],
         [145, 151, 156],
         ...,
         [167, 171, 172],
         [167, 171, 172],
         [167, 171, 172]],
 
        [[145, 151, 156],
         [145, 151, 156],
         [145, 151, 156],
         ...,
         [167, 171, 172],
         [167, 171, 172],
         [167, 171, 172]],
 
        [[145, 151, 156],
         [145, 151, 156],
         [145, 151, 156],
         ...,
         [167, 171, 172],
         [167, 171, 172],
         [167, 171, 172]],
 
        ...,
 
        [[ 74, 125, 175],
         [ 74, 125, 175],
         [ 74, 125, 175],
         ...,
         [ 82, 141, 190],
         [ 82, 141, 190],
         [ 82, 141, 190]],
 
        [[ 74, 125, 175],
         [ 74, 125, 175],
         [ 74, 125, 175],
         ...,
         [ 81, 140

In [8]:
import csv

# Run validation
metrics = model.val()

# Optional: Compute F1 Score
precision = metrics.box.p
recall = metrics.box.r
f1_score = metrics.box.f1

# Define custom row to log
log_data = {
    'model_name': 'baseline_yolov8',
    'epochs': model.args['epochs'],
    'batch': model.args['batch'],
    'lr0': model.args['lr0'],
    'precision': float(precision),
    'recall': float(recall),
    'mAP_0.5': float(metrics.box.map50),
    'mAP_0.5:0.95': float(metrics.box.map),
    'f1_score': float(f1_score)
}

# Write to CSV
csv_file = '/content/runs/detect/baseline_yolov8/yolov8_baseline_eval_log.csv'
fieldnames = list(log_data.keys())

# Append or create new
write_header = not os.path.exists(csv_file)
with open(csv_file, mode='a', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    if write_header:
        writer.writeheader()
    writer.writerow(log_data)

print(f"Logged evaluation metrics to {csv_file}")

Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1686.7±604.8 MB/s, size: 41.0 KB)


val: Scanning /content/data/val/labels.cache... 2151 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2151/2151 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 292, len(boxes) = 8175. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 135/135 [00:11<00:00, 11.28it/s]


                   all       2151       8175      0.946      0.926      0.967      0.764
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/baseline_yolov83
Logged evaluation metrics to /content/runs/detect/baseline_yolov8/yolov8_baseline_eval_log.csv


<ipython-input-8-a1212a0245da>:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'precision': float(precision),
<ipython-input-8-a1212a0245da>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'recall': float(recall),
<ipython-input-8-a1212a0245da>:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'f1_score': float(f1_score)


In [9]:
import json

# Save training args to JSON inside the YOLOv8 output folder
json_path = '/content/runs/detect/baseline_yolov8/baseline_yolov8_args.json'
with open(json_path, 'w') as f:
    json.dump(model.args, f, indent=4)

print(f"Saved model training args to {json_path}")

Saved model training args to /content/runs/detect/baseline_yolov8/baseline_yolov8_args.json


In [13]:
!zip -r /content/baseline_yolov8_results.zip /content/runs/detect/

from google.colab import files
files.download('/content/baseline_yolov8_results.zip')

  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/baseline_yolov83/ (stored 0%)
  adding: content/runs/detect/baseline_yolov83/confusion_matrix.png (deflated 34%)
  adding: content/runs/detect/baseline_yolov83/val_batch2_labels.jpg (deflated 8%)
  adding: content/runs/detect/baseline_yolov83/F1_curve.png (deflated 16%)
  adding: content/runs/detect/baseline_yolov83/val_batch2_pred.jpg (deflated 8%)
  adding: content/runs/detect/baseline_yolov83/val_batch0_pred.jpg (deflated 5%)
  adding: content/runs/detect/baseline_yolov83/confusion_matrix_normalized.png (deflated 36%)
  adding: content/runs/detect/baseline_yolov83/val_batch1_labels.jpg (deflated 5%)
  adding: content/runs/detect/baseline_yolov83/R_curve.png (deflated 16%)
  adding: content/runs/detect/baseline_yolov83/val_batch1_pred.jpg (deflated 5%)
  adding: content/runs/detect/baseline_yolov83/PR_curve.png (deflated 23%)
  adding: content/runs/detect/baseline_yolov83/val_batch0_labels.jpg (deflated 5%)
  ad

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>